In [15]:
# ## Task 1: Mark all launch sites on a map
import folium
import pandas as pd

# Get unique launch sites with coordinates
unique_sites = df[['Launch Site', 'Latitude', 'Longitude']].drop_duplicates().reset_index(drop=True)
# Center the map on the average location
site_map = folium.Map(location=[unique_sites['Latitude'].mean(), unique_sites['Longitude'].mean()],
                      zoom_start=5)
# Add markers for each unique launch site
for _, row in unique_sites.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=row['Launch Site'],
        icon=folium.Icon(color='blue', icon='rocket', prefix='fa')
    ).add_to(site_map)

# Display the map
site_map


In [16]:
# Task 2: Mark the success/failed launches for each site on the map
import numpy as np

# Simulate success/failure labels for visualization (optional)
np.random.seed(42)  # for reproducibility
df['Class'] = np.random.choice([0, 1], size=len(df))  # 0 = failure, 1 = success
# Create map centered at average location
outcome_map = folium.Map(location=[df['Latitude'].mean(), df['Longitude'].mean()], zoom_start=5)

# Plot success and failure
for _, row in df.iterrows():
    color = 'green' if row['Class'] == 1 else 'red'
    folium.CircleMarker(
        location=[row['Latitude'], row['Longitude']],
        radius=6,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.7,
        popup=f"{row['Launch Site']} - {'Success' if row['Class'] == 1 else 'Failure'}"
    ).add_to(outcome_map)

# Show map
outcome_map


In [17]:
#TASK 3: Calculate the distances between a launch site to its proximities
!pip install geopy



   -------------------- ------------------- 1/2 [geopy]
   -------------------- ------------------- 1/2 [geopy]
   -------------------- ------------------- 1/2 [geopy]
   -------------------- ------------------- 1/2 [geopy]
   ---------------------------------------- 2/2 [geopy]



In [18]:
from geopy.distance import geodesic
# Example proximity feature locations
proximity_features = {
    "Coastline": (28.564, -80.568),      # Near CCAFS
    "Railway": (28.572, -80.585),        # Near KSC
    "Highway": (34.636, -120.610),       # Near VAFB
    "City": (28.538336, -81.379234),     # Orlando, FL
}
# Create a set of unique launch site coordinates
unique_sites = df[['Launch Site', 'Latitude', 'Longitude']].drop_duplicates()

# Calculate and store distances
distances = []

for _, site in unique_sites.iterrows():
    site_coord = (site['Latitude'], site['Longitude'])
    site_name = site['Launch Site']
    
    for feature_name, feature_coord in proximity_features.items():
        distance_km = geodesic(site_coord, feature_coord).km
        distances.append({
            'Launch Site': site_name,
            'Feature': feature_name,
            'Distance (km)': round(distance_km, 2)
        })

# Convert to DataFrame
distances_df = pd.DataFrame(distances)
distances_df


,Launch Site,Feature,Distance (km)
0,CCAFS LC-40,Coastline,0.93
1,CCAFS LC-40,Railway,1.31
2,CCAFS LC-40,Highway,3832.99
3,CCAFS LC-40,City,78.51
4,VAFB SLC-4E,Coastline,3833.86
5,VAFB SLC-4E,Railway,3831.99
6,VAFB SLC-4E,Highway,0.36
7,VAFB SLC-4E,City,3760.48
8,KSC LC-39A,Coastline,7.79
9,KSC LC-39A,Railway,6.06
